In [1]:
#r "nuget: BenchmarkDotNet, 0.13.9"
#r "nuget: Microsoft.Playwright, 1.39.0"
#r "bin/Release/net6.0/ConsoleBenchmark.dll"

using ConsoleBenchmark;

Installed Packages BenchmarkDotNet, 0.13.9 Microsoft.Playwright, 1.39.0

In [20]:
var stopWatch = new System.Diagnostics.Stopwatch();
await using (var fixture = await PlaywrightFixture.CreateAsync()){
    stopWatch.Start();
    var jsonElement = await fixture.Expression1JsonElementEvaluateAsync();
    stopWatch.Stop();
}
Console.WriteLine($"{nameof(PlaywrightFixture.Expression1JsonElementEvaluateAsync)}: {stopWatch.ElapsedMilliseconds}ms");

Expression1JsonElementEvaluateAsync: 6ms


In [21]:
var stopWatch = new System.Diagnostics.Stopwatch();
await using (var fixture = await PlaywrightFixture.CreateAsync()){
    stopWatch.Start();
    var gameState = await fixture.Expression2StringDeserializeEvaluateAsync();
    stopWatch.Stop();
}
Console.WriteLine($"{nameof(PlaywrightFixture.Expression2StringDeserializeEvaluateAsync)}: {stopWatch.ElapsedMilliseconds}ms");

Expression2StringDeserializeEvaluateAsync: 6ms


In [3]:
var stopWatch = new System.Diagnostics.Stopwatch();
await using (var fixture = await PlaywrightFixture.CreateAsync()){
    stopWatch.Start();
    var screenshot = await fixture.ScreenshotAsync();
    stopWatch.Stop();
}
Console.WriteLine($"{nameof(PlaywrightFixture.ScreenshotAsync)}: {stopWatch.ElapsedMilliseconds}ms");

ScreenshotAsync: 243ms
